In [ ]:
# Auto reload modules
%load_ext autoreload
%autoreload 2
#if you haven't installed vsf and just want to run this from the current directory, uncomment the following lines
import sys
sys.path.append('..')

### Download the demo data from box (80MB): 
+ #### Demo data: https://uofi.box.com/s/31wozq63qgqvg8012r1jdfj5mdchmmfp

### Visualize VSF

We have already estimated some neural VSFs for you in the demo dataset.  To visualize them, you can use the `vsf_show` function.

In [ ]:
import os

# demo data directory
DEMO_DIR = "../demo_data" # change to your path
OBJECT = "brown_boot_moving" # choose from "brown_boot_fixed" or "brown_boot_moving"
DATA_DIR = os.path.join(DEMO_DIR, 'datasets', OBJECT)
TRIAL = ""

In [ ]:
# visualize VSF
from vsf.constructors import vsf_from_file
import torch

#estimated VSFs have been saved to the "outputs" folder
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vsf = vsf_from_file(os.path.join(DEMO_DIR, "saved_vsfs", OBJECT, "neural_vsf.pt")).to(device)

from klampt import vis
from vsf.visualize.klampt_visualization import vsf_show
vis.init('PyQt')  #needed inside Jupyter Notebook to show an OpenGL window
vsf_show(vsf)
vis.clear()

### Walkthrough neural VSF training and visualization

### Create a world and simulator

In [ ]:
import numpy as np
import json
import klampt
from klampt.math import se3
from vsf.sim import QuasistaticVSFSimulator
from vsf.sensor.punyo_dense_force_sensor import PunyoDenseForceSensor
from vsf.sim.klampt_world_wrapper import klamptWorldWrapper
from vsf.utils.klampt_utils import load_trimesh_preserve_vertices

# NOTE: This part of the code shows how to manually setup a simulation world with a robot, a tactile sensor and a deformable object,
#       and how to estimate the VSF from tactile data.
#       A script to automate this process from config files is in scripts/neural_vsf_estimate.py

# create a world for simulation and visualization
world = klamptWorldWrapper()

# add robot/mesh to the world
world.add_robot('kinova', os.path.join('../knowledge/robot_model', 'kinova_gen3.urdf'))
robot = world.world.robot(0)
ee_link = robot.link(robot.numLinks()-1)
punyo2ee = np.array(json.load(open(os.path.join('../knowledge/robot_model', 'punyo2end_effector_transform.json')))['punyo2ee'])

#this function should be used instead of native Klampt loaders due to a known Assimp configuration issue
mesh = load_trimesh_preserve_vertices(os.path.join('../knowledge/robot_model', 'punyo_mesh_partial.ply'))
world.add_geometry('punyo', mesh, 'deformable', ee_link.getName(), punyo2ee)

# adds the VSF object to the world, for visualization purposes
# hack for add object to the visualization without interfering with the simulator
world2 = klampt.WorldModel()
object = world2.makeRigidObject("object")
if os.path.exists(os.path.join(DATA_DIR, "object", "mesh.obj")):
    object.geometry().loadFile(os.path.join(DATA_DIR, "object", "mesh.obj"))
else:
    aabb = np.load(os.path.join(DATA_DIR, "object", "aabb.npy"))
    object.geometry().setTriangleMesh(klampt.model.create.primitives.bbox(*aabb).getTriangleMesh())

# initialize sensors
sensors = [PunyoDenseForceSensor('punyo', 'punyo')] # add sensor named 'punyo' and attach it to the mesh named 'punyo'

# create simulator
sim = QuasistaticVSFSimulator(world, sensors)

### Load tactile dataset and visualize it

In [ ]:
import dacite
from vsf.dataset.constructors import dataset_from_config, DatasetConfig

from vsf.utils.config_utils import load_config_recursive
config = load_config_recursive(os.path.join('../configs/neural_vsf_dense_forces.yaml'))
dataset_config = dacite.from_dict(DatasetConfig, config['dataset'])
dataset_config.path = os.path.join('../demo_data/datasets', OBJECT, TRIAL)
dataset = dataset_from_config(dataset_config)

In [ ]:
from klampt import vis
vis.init('PyQt')  #needed inside Jupyter Notebook to show an OpenGL window
vis.clear()
vis.add("world", world.world)

# add table
# the table are for visualization only, not added to the simulator to avoid extra computation in the simulation
from klampt.model.create import box
b1 = box(3.0,3.0,1.1,center=(0,0,-0.55),type='GeometricPrimitive')
vis.add("table", b1, hide_label=True)
b2 = box(1.2,1.,0.12,center=(0.5,0.7,0),type='GeometricPrimitive')
vis.add("box", b2, hide_label=True)

# add visualization object to show its estimated pose
vis.add("object", object)
vis.show()

import time
for i in range(len(dataset)):
    seq = dataset[i]
    for frame in seq:
        control = {}
        control['kinova'] = frame['angles']
        control['punyo'] = frame['punyo_deformed']

        # step simulation
        vis.lock()
        sim.step(control, 0.1)
        object.setTransform(*se3.from_ndarray(frame['object_pose'])) # object mesh is for visualization only, so not added to the simulator
        vis.unlock()

        time.sleep(0.1)

    sim.reset()
    if i >= 5:
        break
vis.clear()

In [ ]:
# create VSF model
import torch
from vsf.constructors import vsf_from_box, vsf_from_mesh

# create vsf from a bounding box
# aabb = np.load(os.path.join(DATA_DIR, "object", "aabb.npy"))
# vsf = vsf_from_box(aabb[0], aabb[1], type='neural')

# create vsf from a mesh -- this will do a better job estimating stiffness at the object boundaries
vsf = vsf_from_mesh(os.path.join(DATA_DIR, "object", "mesh.obj"))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vsf = vsf.to(device)

# add vsf to the scene
sim.add_deformable('boot',vsf)

## Run estimation

The following code runs a batch estimation over the whole dataset.  Note that training is performed by randomizing over sequences in the dataset.

In [ ]:
# Batch VSF estimation demo
from vsf.estimator.neural_vsf_estimator import NeuralVSFEstimator, NeuralVSFEstimatorConfig

# create estimator
estimator_config = NeuralVSFEstimatorConfig(lr=2e-4,
                                            regularizer_samples=500,
                                            regularizer_scale=1e-4,
                                            max_epochs=500)
estimator = NeuralVSFEstimator(estimator_config)

print("Starting batch estimation, using device",vsf.device)
estimator.batch_estimate(sim, vsf, dataset, dataset_config)

The following code runs an online estimation for each sequence in the dataset.  This does have a risk of catastrophic forgetting, since there is no memory replay buffer (as in Point VSF online estimators).

In [ ]:
# Online VSF estimation demo

# create estimator
estimator_config = NeuralVSFEstimatorConfig(lr=2e-4,
                                            regularizer_samples=500,
                                            regularizer_scale=1e-4)
estimator = NeuralVSFEstimator(estimator_config)

estimator.online_init(sim, vsf)

sensor_keys = dataset_config.sensor_keys
control_keys = dataset_config.control_keysdt = 0.1

dt = 0.1
for i in range(len(dataset)):
    print("Beginning sequence",i)
    seq = dataset[i]
    sim.reset()
    estimator.online_reset(sim)
    for frame in seq:
        control, observation = {}, {}
        for k in sensor_keys:
            observation[k] = frame[sensor_keys[k]]
        for k in control_keys:
            control[k] = frame[control_keys[k]]

        sim.step(control, dt)
        loss = estimator.online_update(sim, dt, observation)
        print("Loss",loss)


In [ ]:
# visualize VSF
from vsf.visualize.klampt_visualization import vsf_show
vsf_show(vsf)

In [ ]:
# save to disk        
os.makedirs(os.path.join(DEMO_DIR, "outputs", OBJECT), exist_ok=True)
vsf.save(os.path.join(DEMO_DIR, "outputs", OBJECT, "neural_vsf_playground.pt"))


### Neural VSF to Point VSF


In [ ]:
from vsf.constructors import vsf_from_vsf, PointVSFConfig

voxel_size = 3e-3
point_config = PointVSFConfig(voxel_size=voxel_size)
point_vsf = vsf_from_vsf(point_config, vsf)
vsf_show(point_vsf)

### Point VSF to Neural VSF

In [ ]:
from vsf.constructors import vsf_from_vsf, NeuralVSFConfig

neural_config = NeuralVSFConfig()
neural_vsf = vsf_from_vsf(neural_config, point_vsf)
vsf_show(neural_vsf)